<a href="https://www.kaggle.com/code/sivamanivallabhani/bertfinetuning?scriptVersionId=175454782" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Regular imports
import matplotlib.pyplot as plt
import torch
from torch import nn
import numpy as np 
device = "cuda" if torch.cuda.is_available() else "cpu"

# Import torch.nn.functional as F
import torch.nn.functional as F

In [2]:
# Define data path
data_path = "/kaggle/input/davidson1/labeled_data.csv"


In [3]:
# Load data
import pandas as pd
data = pd.read_csv(data_path)
df = pd.DataFrame(data)
df.columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
              'classk', 'tweet']
df["classk"] = df["classk"].map({2:1, 1:0, 0:0})  # Convert labels to binary (hate vs not hate)
tweets = df.tweet
res = []


In [4]:
# Process tweets
for i in range(len(tweets)):
    splitt = tweets[i].split(":")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])

tweets = res

In [5]:

# Import tokenizer and BERT model
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# Define custom dataset class
class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int):
        X = self.data[index]
        y = self.labels[index]
        return (X, y)
    
    def __len__(self) -> int:
        return len(self.data)


In [7]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

# Define train and test datasets and dataloaders
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [8]:
class Model0(nn.Module):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
        self.fc1 = nn.Linear(768, 256)  # Additional layer for fine-tuning
        self.fc2 = nn.Linear(256, 2)     # Output layer

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][0].unsqueeze(1)

        
        # Additional layers for fine-tuning
        outputs = torch.relu(self.fc1(outputs.squeeze()))  # Remove extra dimension
        outputs = self.fc2(outputs)

        return outputs


In [9]:
# Instantiate the model
model0 = Model0(model, tokenizer)
model0.to(device)

# Define the optimizer
optimizer = torch.optim.Adam(params=model0.parameters(), lr=0.00002)

In [10]:
# Define the inter and intra space loss functions
def L_inter(spaces):
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        loss = loss + cur
    return loss

def L_intra(spaces):
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss = loss + 1 / torch.var(spaces[k], dim=0)
    return torch.sum(loss) / loss.shape[0]

In [11]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
#         if i == 100 :
#             break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.16844865679740906


In [12]:
# Train the model
model0.train()
try:
    for _ in range(3):
        for i, batch in enumerate(train_loader):
#             if i==100:
#                 break
            if i % 2000 == 0:
                print(i)

            X, Y = batch
            loss = 0

            for X, y in zip(X, Y):
                y_preds = model0(X)
                loss += F.cross_entropy(y_preds.view(1, -1).to(device), torch.tensor([y]).to(device))

            loss /= len(Y)
#             loss += 0.7 * L_inter(torch.cat((hate_space, not_hate_space))) + 0.5 * L_intra(torch.stack([hate_space, not_hate_space]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        

except KeyboardInterrupt:
    print("Training interrupted by the user.")

0
2000
4000
6000
0
2000
4000
6000
0
2000
4000
6000


In [13]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
#         if i == 100 :
#             break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.9314101338386536


In [14]:
# Define hate and not_hate words
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 
              'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
                  'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [15]:
temp = []
for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

hate_space = torch.tensor(temp).to(device).requires_grad_()

# Process not_hate words
temp = []
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

not_hate_space = torch.tensor(temp).to(device).requires_grad_()

In [16]:
import random
l=[[random.uniform(0,1)] for i in range(11)]
l

[[0.938479864830867],
 [0.9908313225161386],
 [0.40252233352843214],
 [0.4822006537085368],
 [0.552584745803621],
 [0.015420599293426096],
 [0.5335351880818278],
 [0.9753431287025417],
 [0.5223441222835631],
 [0.9280562599056817],
 [0.7098964017919168]]

In [17]:
# next(iter(test_loader)) 
tensor_shape = (11,1)
hate_space_weights = torch.tensor(l).to(device).requires_grad_()
not_hate_space_weights = torch.tensor(l).to(device).requires_grad_()

print(hate_space_weights.device,not_hate_space_weights.is_leaf)

cuda:0 True


In [18]:
hate_space_weights.shape

torch.Size([11, 1])

In [19]:
hate_space_weights.is_leaf

True

In [20]:
import torch.nn.functional as F

class Model1(nn.Module):

    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])].transpose(0, 1)#shape=>(768,no of tokens)
        col_norms = torch.norm(outputs, p=2, dim=0, keepdim=True)
        outputs = outputs / col_norms#makes every word resp unit vector 
        hate_att_scores=torch.matmul(hate_space/torch.norm(hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        not_hate_att_scores=torch.matmul(not_hate_space/torch.norm(not_hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        Psk_hate = torch.matmul(hate_att_scores.mean(1), hate_space_weights)#shape=>(1)
        Psk_not_hate = torch.matmul(not_hate_att_scores.mean(1), not_hate_space_weights)#shape=>(1)
        output_tensor = torch.cat((Psk_hate, Psk_not_hate), dim=0).requires_grad_()#
        return output_tensor

In [21]:
model1 = Model1(model, tokenizer)

In [22]:
def L_inter(spaces):
    """this function calculates inter space losses between spaces 
    forces the class spaces of different class to move apart
    """
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    """"this function calculates intra space loss 
forces the vector representations inside a class space to have high variability(varience)
"""
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]

In [23]:
print(hate_space,not_hate_space)

tensor([[-0.0525,  0.5070,  0.0941,  ...,  0.1657,  0.0412, -0.2926],
        [ 0.3073,  1.4053, -0.9369,  ..., -0.8997, -0.0616,  0.6982],
        [ 0.2400,  1.6331, -0.3888,  ..., -0.7525, -0.1699,  0.0657],
        ...,
        [ 0.1618,  1.4558, -0.4295,  ..., -0.6166, -0.0106,  0.6783],
        [ 0.2406,  1.1614, -0.7776,  ..., -0.6917, -0.2472,  0.5381],
        [ 0.3051,  1.8673, -0.6012,  ..., -0.3874, -0.0905,  0.4414]],
       device='cuda:0', requires_grad=True) tensor([[-0.0939,  0.3199,  0.4036,  ...,  0.3759, -0.0233, -0.0030],
        [-0.4609,  0.1847,  0.1065,  ...,  0.6008,  0.2508, -0.3878],
        [ 0.1257,  0.0428,  0.1504,  ...,  0.6021, -0.0594, -0.2107],
        ...,
        [-0.0695,  0.2656, -0.0764,  ...,  0.4471, -0.1805, -0.2571],
        [ 0.2441,  0.4750,  0.1748,  ...,  0.1581,  0.0961, -0.1102],
        [ 0.2220,  0.4994, -0.0185,  ..., -0.0397, -0.2848, -0.5218]],
       device='cuda:0', requires_grad=True)


In [24]:
print(hate_space_weights,not_hate_space_weights)

tensor([[0.9385],
        [0.9908],
        [0.4025],
        [0.4822],
        [0.5526],
        [0.0154],
        [0.5335],
        [0.9753],
        [0.5223],
        [0.9281],
        [0.7099]], device='cuda:0', requires_grad=True) tensor([[0.9385],
        [0.9908],
        [0.4025],
        [0.4822],
        [0.5526],
        [0.0154],
        [0.5335],
        [0.9753],
        [0.5223],
        [0.9281],
        [0.7099]], device='cuda:0', requires_grad=True)


In [25]:
from sklearn.metrics import precision_score, recall_score,accuracy_score


model1.to(device)

print(hate_space.is_leaf)
optimizer = torch.optim.Adam(params=[hate_space, not_hate_space, hate_space_weights, not_hate_space_weights], lr=0.0002)
model1.train()


curr=0.1
prev=0
pprev=0
for _ in range(5):
    for i, batch in enumerate(train_loader):
#         if i == 100:
#             break
        if i % 2000 == 0:
            print(i)

        x, Y = batch
        loss = 0

        for X, y in zip(x, Y):
            probs = model1(X)
#             print(y_preds)
            #print(f"printing y_preds={y_preds} shape={y_preds.shape}")

            loss+=F.cross_entropy(probs.view(1, -1).to(device), torch.tensor([y]).to(device))
            """explisit soft max is not used to convert to probs is cross 
            entrophy loss has inbuilt softmax  function adding softmax explisity
            will effect adversly by smoothing again on smoothed values will decrese 
            the weightage for class with high prob """
#             break
        loss /= len(x)
        loss += 1*L_inter(torch.cat((hate_space, not_hate_space))) + 1*L_intra(torch.stack([hate_space, not_hate_space]))
            
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
#         print(hate_space.grad,hate_space_weights.grad)
    correct = 0
    total = len(y_test)

    true = []
    preds = []

    with torch.inference_mode():
        for i, batch in enumerate(test_loader):
#             if i==100:
#                 break

            X, y = batch
            Psk_hate, Psk_not_hate = model1(X)

            y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

            true.extend(y)
            preds.append(y_predict)

            cur = torch.eq(y_predict, y)

            correct += torch.sum(cur)
    pprev=prev
    prev=curr
    print(f"Test Accuracy: {correct/total}")
    curr=correct/total
    print(f"Accuracy - {accuracy_score(y_test, preds)}")
    print(f"precison score -{precision_score(y_test, preds, average='macro')}")
    print(f"recall score  -{recall_score(y_test, preds, average='macro')}")
  

True
0
2000
4000
6000
Test Accuracy: 0.9277788996696472
Accuracy - 0.9277788985273351
precison score -0.8721494997419397
recall score  -0.8671134258380748
0
2000
4000
6000
Test Accuracy: 0.9297962188720703
Accuracy - 0.9297962477304822
precison score -0.8792535078206402
recall score  -0.8644749095728687
0
2000
4000
6000
Test Accuracy: 0.9304014444351196
Accuracy - 0.9304014524914263
precison score -0.8840453961482813
recall score  -0.8595445508712826
0
2000
4000
6000
Test Accuracy: 0.929997980594635
Accuracy - 0.9299979826507968
precison score -0.8814819604293289
recall score  -0.8617085254027985
0
2000
4000
6000
Test Accuracy: 0.9287875890731812
Accuracy - 0.9287875731289086
precison score -0.8777837704423501
recall score  -0.8619441176384725


In [26]:
print(hate_space_weights,not_hate_space_weights)

tensor([[ 1.2160],
        [ 0.8854],
        [ 0.4649],
        [ 0.2494],
        [ 0.7788],
        [-0.1880],
        [ 0.4167],
        [ 0.8205],
        [ 0.3139],
        [ 0.8044],
        [ 0.7191]], device='cuda:0', requires_grad=True) tensor([[ 0.8511],
        [ 0.7731],
        [ 0.1595],
        [ 0.2863],
        [ 0.3622],
        [-0.0559],
        [ 0.4556],
        [ 0.8731],
        [ 0.3371],
        [ 0.8152],
        [ 0.6101]], device='cuda:0', requires_grad=True)


In [27]:
print(hate_space,not_hate_space)

tensor([[-0.3188, -0.6435,  1.3017,  ...,  1.3259,  0.4640, -2.2231],
        [ 2.1198,  2.6617, -2.1869,  ..., -2.0026,  1.7785,  1.6407],
        [-1.2765,  3.8438,  1.0696,  ..., -1.4766, -2.5157, -2.4781],
        ...,
        [-1.2639,  5.3552,  1.2404,  ..., -5.4321,  1.4033,  3.3653],
        [-0.7008,  0.0122, -2.1947,  ..., -2.1338, -1.5813,  0.8939],
        [ 2.6994,  4.0545,  0.8594,  ...,  1.0566, -2.6235,  0.9603]],
       device='cuda:0', requires_grad=True) tensor([[-1.5878,  2.0495,  1.7986,  ...,  1.8380, -1.3549,  1.7028],
        [-1.9368, -0.8519, -1.3041,  ...,  1.8883,  2.2620, -1.2161],
        [ 5.0420, -2.3547, -3.2843,  ...,  4.4718, -4.5288, -4.3078],
        ...,
        [-2.6763, -1.8471, -1.7031,  ...,  4.3592, -2.4898, -2.9236],
        [ 1.3113,  1.7434,  2.3881,  ..., -1.0373,  2.4028,  1.7769],
        [ 1.6071,  2.0615, -1.1512,  ..., -1.2673, -1.3461, -1.3746]],
       device='cuda:0', requires_grad=True)


In [28]:
def save_bert_model(model,tokenizer,params):
        output_dir = 'Saved/bert'
#         if(params['train_att']):
#             if(params['lambda_attn']>=1):
#                 params['lambda_attn']=int(params['lambda_attn'])

#             output_dir =  output_dir+str(params['supervised_layer_pos'])+'_'+str(params['num_supervised_heads'])+'_'+str(params['num_classes'])+'_'+str(params['lambda_attn'])+'/'
            
#         else:
#             output_dir=output_dir+'_'+str(params['num_classes'])+'/'
#         print(output_dir)
#         # Create output directory if needed
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

In [29]:
save_bert_model(model,tokenizer,model.parameters())

In [30]:
torch.save(hate_space, 'hatespace.pt')
torch.save(hate_space_weights, 'hatespace.pt_weights')
torch.save(not_hate_space, 'not_hatespace.pt')
torch.save(not_hate_space_weights, 'not_hatespace_weights.pt')